# Conditional Prompt Selector

LLM will not be invoked. They are referenced to demonstrate the working of the conditional prompting.

https://api.python.langchain.com/en/stable/chains/langchain.chains.prompt_selector.ConditionalPromptSelector.html#

#### Google Colab
If you are running the code in Google colab, install the packages by uncommenting/running the cell below

Uncomment & run the code in the cell below:

In [ ]:
# !pip install langchain langchain-cohere>=0.2.2 -q

## Import the required packages

In [1]:
from langchain import PromptTemplate
from langchain.chains.prompt_selector import ConditionalPromptSelector
import warnings

warnings.filterwarnings("ignore")

## 1. Define the default template

In [2]:
default_prompt = """summarize the text below.

Text:
{text}"""

default_prompt_template = PromptTemplate(template=default_prompt, input_variables=["text"])

# default_prompt.format(text="DUMMY")

## 2. AI21 prompt template &LLM

**ASSUME** that following prompt produced a better result compared to the default prompt.




In [3]:
ai21_prompt = """
<<SYS>>
You are an assistant tasked with summarizing technical research papers.
<</SYS>> 

[INST] 
Generate summary: 

{text} 
[/INST]"""

ai21_prompt_template = PromptTemplate(template=ai21_prompt, input_variables=["text"])

# ai21_prompt_template.format(text="DUMMY")

In [4]:
from langchain_community.llms.ai21 import AI21

ai21_llm = AI21(ai21_api_key="DUMMY-KEY")

## 3. Cohere Command prompt template & LLM

**ASSUME** that following prompt produced a better result compared to the default prompt.

In [5]:
cohere_prompt = """
You are an expert technical writer. Your task is to summarize the given research paper's content.

Content:
{text}"""

cohere_prompt_template = PromptTemplate(template=cohere_prompt, input_variables=["text"])

# cohere_prompt_template.format(text="DUMMY")

In [6]:
from langchain_community.llms.cohere import Cohere

cohere_llm = Cohere(cohere_api_key="DUMMY-KEY")

C:\Users\raj\anaconda3\envs\gen-ai-app-dev-course\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(
C:\Users\raj\anaconda3\envs\gen-ai-app-dev-course\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


## 4. Create the conditional prompt

Provide instance of LLM and 

In [7]:

conditional_prompt_selector = ConditionalPromptSelector(
    default_prompt=  default_prompt_template,
    conditionals=[
        (lambda llm: isinstance(llm, type(ai21_llm)), ai21_prompt_template),
        (lambda llm: isinstance(llm, type(cohere_llm)), cohere_prompt_template)
    ]
)


## 5. Test the prompt generator

In [8]:
# Using AI21 model
llm = ai21_llm
prompt = conditional_prompt_selector.get_prompt(llm)

print(prompt.format(text="DUMMY"))


<<SYS>>
You are an assistant tasked with summarizing technical research papers.
<</SYS>> 

[INST] 
Generate summary: 

DUMMY 
[/INST]


In [9]:
# Using AI21 model
llm = cohere_llm
prompt = conditional_prompt_selector.get_prompt(llm)

print(prompt.format(text="DUMMY"))


You are an expert technical writer. Your task is to summarize the given research paper's content.

Content:
DUMMY


In [ ]:
## Now you can use the llm & prompt to carry out the infrence
# llm.invoke(prompt)